In [36]:
import pandas as pd
from database.database import Database
from modeler.modeler import Modeler as m
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from textblob import TextBlob
from sklearn.metrics import r2_score

In [2]:
project_name = "youtube_trends"

In [3]:
db = Database(project_name)

In [4]:
## Transform data
data = pd.read_csv("./data/US_youtube_trending_data.csv")

In [10]:
data["tag"] = [x.split("|")[0] for x in data["tags"]]

In [16]:
data["tbtitlepol"] = [TextBlob(str(x)).sentiment.polarity for x in data["title"]]
data["tbtitlesub"] = [TextBlob(str(x)).sentiment.subjectivity for x in data["title"]]

TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'float'>

In [17]:
data["tbdescriptionpol"] = [TextBlob(str(x)).sentiment.polarity for x in data["description"]]
data["tbdescriptionsub"] = [TextBlob(str(x)).sentiment.subjectivity for x in data["description"]]

In [18]:
data["tbtagpol"] = [TextBlob(str(x)).sentiment.polarity for x in data["tag"]]
data["tbtagsub"] = [TextBlob(str(x)).sentiment.subjectivity for x in data["tag"]]

In [50]:
factors = [x for x in data.columns if "tb" in x]

In [24]:
data["like_pct"] = data["likes"]  / (data["likes"] + data["dislikes"])
data["publishedAt"] = pd.to_datetime(data["publishedAt"])
data["trending_date"] = pd.to_datetime(data["trending_date"])
data["daystotrending"] = data["trending_date"] - data["publishedAt"]
data["daystotrending"] = [x.days for x in data["daystotrending"]]

In [49]:
# data["publish_month"] = [x.month for x in data["publishedAt"]]
# data["publish_week"] = [x.week for x in data["publishedAt"]]

In [51]:
factors.append("like_pct")
factors.append("view_count")
# factors.append("publish_month")
# factors.append("publish_week")

In [52]:
## Refine Data
## if classification make appropriate changes
label = "daystotrending"
data["y"] = data[label]
data.drop(label,axis=1)
data = data.sample(frac=1)
train = data.iloc[:int(data.index.size * 0.7)]
test = data.iloc[int(data.index.size * 0.7):]
X = train[factors]
y = train[["y"]]
refined_data = {"X":X,"y":y}

In [53]:
## Model
model = m.xgb_model(refined_data)

In [54]:
model

{'api': 'xgb',
 'model': XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None),
 'score': -1.1893420267229398}

In [ ]:
## Test Model
predictions = [x[0] for x in model["model"].predict(test[factors])]
test["predictions"] = predictions

In [ ]:
## Visualize
plt.plot(test[index],test["predictions"])
plt.plot(test[index],test[label])

In [ ]:
## Store Model
db.connect()
db.store_data("model",model)
db.close()